# XML



Из документации к SQL Server:
### Модель данных: реляционная или XML

Если данные хорошо структурированы и известна их схема, то для их хранения лучше всего подойдет реляционная модель. 

С другой стороны, если данные структурированы частично, не структурированы или если их структура неизвестна, следует подумать о моделировании таких данных.

XML является удачным выбором, если 
- нужна не зависящая от платформы модель, позволяющая гарантировать совместимость данных;
- данные разрежены, их структура неизвестна или их структура может значительно измениться в будущем;
- данные представляют иерархию контейнеров, а не ссылки между сущностями, и могут быть рекурсивными;
- требуется запрашивать данные или обновлять их фрагменты на основе их структуры.


### XML-тип

In [1]:
declare @xml xml
set @xml = '<root/>'
select @xml
go

set nocount on

declare @xml xml
set @xml = '<root> </root>'
select @xml
go

declare @xml xml
set @xml = 'Text'
select @xml
go

declare @xml xml
set @xml = '<root><element Attr="Text" /></root>'
select @xml
go

declare @xml xml
set @xml = '<root><element Attr=''Text'' /></root>'
select @xml
go

declare @xml xml
set @xml = '
<root>
    <element Attr=''Text1'' />
    <element Attr=''Text2''>
        <subelement>
            Text3
        </subelement>
    </element>
</root>
'
select @xml
go

(1 row affected)

Total execution time: 00:00:00.0062984

Commands completed successfully.

Total execution time: 00:00:00.0037364

Commands completed successfully.

...

(No column name)
<root />


(No column name)
Text


(No column name)
"<root><element Attr=""Text"" /></root>"


(No column name)
"<root><element Attr=""Text"" /></root>"


(No column name)
"<root><element Attr=""Text1"" /><element Attr=""Text2""><subelement> Text3 </subelement></element></root>"


<b>Метод query()</b> (XPath как подмножество XQuery):


In [3]:
declare @xml xml
set @xml = '
<root>
    <element Attr=''Text1'' />
    <element Attr=''Text2''>
        <subelement>
            Text3
        </subelement>
    </element>
</root>
'
select @xml.query('//root/element[@Attr="Text1"]') with_attr1, 
       @xml.query('//element/subelement')          with_subelement,
       @xml.query('//element/subelement/text()')   subelement_text



(1 row affected)

Total execution time: 00:00:00.0233780

with_attr1,with_subelement,subelement_text
"<element Attr=""Text1"" />",<subelement> Text3 </subelement>,Text3


<b>Метод exist():</b>

In [6]:
declare @xml xml
set @xml = '
<root>
    <element Attr=''Text1'' />
    <element Attr=''Text2''>
        <subelement>
            Text3
        </subelement>
    </element>
</root>
'
select @xml, @xml.exist('subelement'), @xml.exist('//root/*/subelement'), @xml.exist('//root/element[@Attr]')




(1 row affected)

Total execution time: 00:00:00.0047927

(No column name),(No column name),(No column name),(No column name)
"<root><element Attr=""Text1"" /><element Attr=""Text2""><subelement> Text3 </subelement></element></root>",0,1,1


<b>Загрузка данных из внешнего XML + метод value():</b>

In [3]:
declare @xml xml

select @xml = BulkColumn    
from openrowset(bulk N'c:/temp/SwimmingCompetition.xml', single_blob) q

--метод value для извлечения данных из XML "наружу":
select @xml.value(N'(//competitions/row/result/athlete_name)[1]','nvarchar(100)')

--альтернативный подход:
select cast(@xml.query(N'(//competitions/row/result/athlete_name)[1]/text()') as nvarchar(100))



(1 row affected)

(1 row affected)

Total execution time: 00:00:00.4215206

(No column name)
САФОНОВА Анастасия


(No column name)
САФОНОВА Анастасия


<b>Метод node():</b>

In [17]:
declare @xml xml

select @xml = BulkColumn    
from openrowset(bulk N'c:/temp/SwimmingCompetition.xml', single_blob) q

--имена, год рождения и клуб всех чемпионов + общее число золотых медалей
;with cte_champions
as
(
    select  
        --xml_col.query('.') xml_col,
        xml_col.value('./athlete_name[1]', 'nvarchar(30)') athlete_name,        
        xml_col.value('./birth_year[1]', 'smallint') birth_year,
        xml_col.value('./team[1]', 'nvarchar(255)') team
    from @xml.nodes(N'//competitions/row[result[place=1]]/result') xml_query(xml_col)
)
select athlete_name, birth_year, team, count(1) cnt
from cte_champions
group by athlete_name, birth_year, team
order by 4 desc




(38 rows affected)

Total execution time: 00:00:01.5280444

athlete_name,birth_year,team,cnt
ШКУРДАЙ Анастасия,2003,"ЦОР-1, Брест",7
РЫЖКОВА Виктория,2006,"СШОР №2, Сызрань",6
САВКИНА Виктория,2005,"Тверская область, Тверь",5
САФОНОВА Анастасия,2005,"ЦЕНТР, Сергиев Посад",5
КРЮК Федор,2004,Москва,4
ПАВЛОВСКИЙ Степан,2004,"СШ № 4, Москва",4
АДАМЧУК Иван,2003,"ЦОР-1, Брест",4
БЕСПАЛОВ Вадим,2005,"СДЮСШОР ЮМ, Москва",4
ДАВЫДОВ Дмитрий,2004,"ОЛИМП, Обнинск",3
ВАСИНА Евгения,2005,"МАУ ДО 'ДЮСШ', Энгельс",3


<b>XPath - выборка:</b>

In [1]:
declare @xml    xml

select @xml = BulkColumn    
from openrowset (bulk N'c:/temp/SwimmingCompetition.xml', single_blob)  q

--Список всех призеров
select @xml.query(N'distinct-values(//competitions/row[result[place<=3]]/result/athlete_name)')

--Сколько раз YERMISHYNA Yelizaveta была в призерах:
select @xml.query(N'count(//competitions/row[result[place<=3 and athlete_name="YERMISHYNA Yelizaveta"]])')

--В каких турнирах участвовала САФОНОВА Анастасия
select @xml.query(N'distinct-values(//competitions/row[result/athlete_name="САФОНОВА Анастасия"]/tournament/name)')



(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:01.8924927

(No column name)
ИЛЬИНСКАЯ Кристина ВОРОНИНА Ксения ГОРОДНЯ Мария ДМИТРИЕВА Мария ПОМЫСОВА Арина ВОРОНКОВА Екатерина ВАСИНА Евгения СЕДАЧЁВА Ульяна СУСОРОВА Софья ОСЬМИНКИНА Таисия YERMISHYNA Yelizaveta КАЙМОНОВА Дарья КОТЫЛЕВСКАЯ Диана САФОНОВА Анастасия ГРИЦУК Анастасия СУЛТАНОВА Анна ГОЛОЩАПОВА Алла ОЛЬШЕВСКАЯ Анастасия НАУМИК Елизавета ШКУРДАЙ Анастасия ЗАКИРОВА Азалия МЕЛЬНИКОВА Елизавета КАНЕВА Анастасия МАТВЕЕВА Светлана РЫБАКОВА Екатерина ЖДАНОВА Ульяна ЛЕТЯГИНА Диана СЕВЕРИН Анна ШОСТАК Анастасия ДЕМЬЯНЕНКО Дарья БЕЛОУСЕНКО Максим БЕСПАЛОВ Вадим ГОФМАН Максим РЫЖОВ Иван ТОКАРЧУК Кирилл СКОРОМНОВ Антон ДАНИЛЬЧЕНКО Лука СНЕТКОВ Никита КИТАЕВ Павел ФИЛИПОВИЧ Филипп ТУРЫШЕВ Егор ГОРНОСТАЕВ Алексей KOCHU Anton ПУШКИН Роман КИСЛОВ Александр ЦЫДЫПОВ Владислав ДЕДОВ Егор ПЕТРОЧУК Iлля НИКИТИН Иван АНДРУШКО Никита АВЕТИСЯН Микаэль АДАМЧУК Иван ТКАЧЁВ Алексей СУШКО Тимур ВОРОБЕЙЧУК Тимофей КУЛИКОВ Максим ДАВЫДОВ Дмитрий БЫКОВ Юрий КАЗАКОВ Никита САБИРОВ Илья ГОРБАЧ Юлия КУРИЛКИНА Александра ПЧЕЛИНА Елизавета ХАЙЛОВА Александра КОЛЕСНИКОВА Таисия ВОЛКОВА Юлия МАТВЕЙЧИК Диана САВКИНА Виктория РОЗОВА Алена ВЛАСОВА Дарья ПЕТРЕНКО София ТАРАСОВА Полина АРТЕМЬЕВА Вероника НЕКРАСОВ Александр ЧУЛКОВ Андрей ЛЕНСКИЙ Егор МАМОНОВ Никита ТКАЧЕВ Алексей ЖУКОВ Илья ЩЕГОЛЕВ Александр БУТКО Иван БОРОДИН Илья КОВАЛЕНКО Алексей БЫСТРОВ Никита КРИВЦОВА Софья ФАДЕЕВА Полина РЫЖКОВА Виктория КУЗНЕЦОВА Александра НАГОВИЦЫНА Мария БЕРЕБНЁВА Варвара ЦАПКИНА Екатерина КАЛАШНИКОВА Дарья ИЩИШЕНА Елизавета ГРИГОРОВИЧ Софья ШУКАЛОВИЧ Кристина ПАВЛОВСКИЙ Степан КРЮК Федор ВИНОГРАДОВ Дмитрий ДЕНЬЩИКОВ Павел ПИСАРЕНКО Артём ВАСИЛЬЕВ Дмитрий ЛИТВИНОВ Кирилл ЗЫРЯНОВ Федор ДЬЯКОВ Александр КОРНЕВ Егор ЗАГЛОДИН Артем КРЮК Борис


(No column name)
2


(No column name)
Minsk Swimming Cup 2018


<b>XPath - вставка:</b>

In [5]:
declare @xml    xml

select @xml = BulkColumn    
from openrowset (bulk N'c:/temp/SwimmingCompetition.xml', single_blob)  q

set @xml.modify(N'insert <gender hack="1">female</gender> into (//competitions/row/group)[1]')

select @xml.query(N'(//competitions/row/group)[position()<3]')



(1 row affected)

Total execution time: 00:00:00.6521286

(No column name)
"<group><athlete_group>2005 и моложе</athlete_group><gender hack=""1"">female</gender></group><group><athlete_group>2005 и моложе</athlete_group></group>"


<b>XPath - удаление:</b>

In [7]:
declare @xml    xml

select @xml = BulkColumn    
from openrowset (bulk N'c:/temp/SwimmingCompetition.xml', single_blob)  q

select @xml.query(N'(//competitions/row)[1]')

set @xml.modify( N'delete (//competitions/row/tournament/name)[1]')

select @xml.query(N'(//competitions/row)[1]')



(1 row affected)

(1 row affected)

Total execution time: 00:00:00.7021526

(No column name)
"<row><tournament><name>Minsk Swimming Cup 2018</name><start_date>2018-07-20</start_date></tournament><pool><name>Dvorets Sporta Main Pool</name><city>Minsk</city><pool_size>50</pool_size></pool><discipline><distance>50</distance><style>Вольный стиль</style></discipline><group><athlete_group>2005 и моложе</athlete_group></group><result><place>1</place><athlete_name>САФОНОВА Анастасия</athlete_name><birth_year>2005</birth_year><team>ЦЕНТР, Сергиев Посад</team><result_time>00:00:30.09</result_time><athlete_rank>2</athlete_rank><points>100</points></result></row>"


(No column name)
"<row><tournament><start_date>2018-07-20</start_date></tournament><pool><name>Dvorets Sporta Main Pool</name><city>Minsk</city><pool_size>50</pool_size></pool><discipline><distance>50</distance><style>Вольный стиль</style></discipline><group><athlete_group>2005 и моложе</athlete_group></group><result><place>1</place><athlete_name>САФОНОВА Анастасия</athlete_name><birth_year>2005</birth_year><team>ЦЕНТР, Сергиев Посад</team><result_time>00:00:30.09</result_time><athlete_rank>2</athlete_rank><points>100</points></result></row>"


<b>XPath - обновление:</b> (+sql:variable("@var"))

In [8]:
drop table if exists #swimming_results

create table #swimming_results (
    id          int     not null identity, 
    xml_value   xml 
)

insert into #swimming_results(xml_value)
select BulkColumn    
from openrowset (bulk N'c:/temp/SwimmingCompetition.xml', single_blob)  q

declare @name_to_replace      nvarchar(100) = N'Dvorets Sporta Main Pool',
        @name_to_replace_with nvarchar(100) = N'Дворец Спорта, основной бассейн'

update #swimming_results 
  set xml_value.modify(
     N'replace value of 
     (//competitions/row/pool/name[text()=sql:variable("@name_to_replace")]/text())[1] 
     with 
     sql:variable("@name_to_replace_with")'
  )

select xml_value.query(N'(//competitions/row/pool)[position()<=2]')
from #swimming_results




(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:01.0322090

(No column name)
"<pool><name>Дворец Спорта, основной бассейн</name><city>Minsk</city><pool_size>50</pool_size></pool><pool><name>Dvorets Sporta Main Pool</name><city>Minsk</city><pool_size>50</pool_size></pool><pool><name>Dvorets Sporta Main Pool</name><city>Minsk</city><pool_size>50</pool_size></pool>"


<b>Использование FLOWR для трансформации XML документа</b> (еще одно подмножество языка XQuery):

In [3]:
declare @xml xml,
        @athlete_name varchar(50) = N'YERMISHYNA Yelizaveta'

set @xml = (select * from openrowset (bulk N'c:/temp/SwimmingCompetition.xml', single_blob)  q)

select @xml.query(
N'
<doc>
{
    for $row in //competitions/row
    where $row/result/athlete_name = sql:variable("@athlete_name")
    return
      <record tournament_name="{$row/tournament/name}" date="{$row/tournament/start_date}" pool_size="{$row/pool/pool_size}">
          <person name="{$row/result/athlete_name}" birth_year="{$row/result/birth_year}" team="{$row/result/team}" />
          <competition style="{$row/discipline/style}" distance="{$row/discipline/distance}" group="{$row/group/athlete_group}" />
          <result time="{$row/result/result_time}" rank="{$row/result/athlete_rank}" points="{$row/result/points}" />
      </record>
}
</doc>
'
)



(1 row affected)

Total execution time: 00:00:00.8855445

(No column name)
"<doc><record tournament_name=""Minsk Swimming Cup 2018"" date=""2018-07-20"" pool_size=""50""><person name=""YERMISHYNA Yelizaveta"" birth_year=""2007"" team=""Dolphin, KIEV"" /><competition style=""Вольный стиль"" distance=""50"" group=""2005 и моложе"" /><result time=""00:00:30.27"" rank=""2"" points=""80"" /></record><record tournament_name=""Minsk Swimming Cup 2018"" date=""2018-07-20"" pool_size=""50""><person name=""YERMISHYNA Yelizaveta"" birth_year=""2007"" team=""Dolphin, KIEV"" /><competition style=""Вольный стиль"" distance=""100"" group=""2005 и моложе"" /><result time=""00:01:07.22"" rank=""1"" points=""60"" /></record><record tournament_name=""Minsk Swimming Cup 2018"" date=""2018-07-20"" pool_size=""50""><person name=""YERMISHYNA Yelizaveta"" birth_year=""2007"" team=""Dolphin, KIEV"" /><competition style=""Баттерфляй"" distance=""50"" group=""2005 и моложе"" /><result time=""00:00:33.39"" rank=""1"" points=""90"" /></record><record tournament_name=""Minsk Swimming Cup 2018"" date=""2018-07-20"" pool_size=""50""><person name=""YERMISHYNA Yelizaveta"" birth_year=""2007"" team=""Dolphin, KIEV"" /><competition style=""Комплекс"" distance=""100"" group=""2005 и моложе"" /><result time=""00:01:17.35"" rank=""б/р"" points=""40"" /></record></doc>"
